<img src="https://upload.wikimedia.org/wikipedia/commons/c/c7/HEIG-VD_Logo_96x29_RVB_ROUGE.png" alt="HEIG-VD Logo" width="250"/>

# TB - DevOps: Mise en place de cloud-native storage
## fio benchmark
**Note**: La version de fio utilisée pour chaque benchmark va être explicitement décrite tel que la license morale (section 4 [documentation](https://fio.readthedocs.io/en/latest/fio_doc.html#moral-license) fio) nous recommande.

## Local
Les résultats de cette section ont été généré à l'aide de fio version 3.16 à l'aide du script `run-all-jobs.sh`.

In [1]:
import json
# from: https://stackoverflow.com/a/3207973
from os import listdir
from os.path import isfile, join

fio_jobs_path = 'fio-jobs/local'
job_outputs_filename = [f for f in listdir(fio_jobs_path) if isfile(join(fio_jobs_path, f)) and 'output' in f]

FileNotFoundError: [Errno 2] No such file or directory: 'fio-jobs/local'

In [ ]:
job_outputs = []

# Load the results
for job_output in job_outputs_filename:
    f = open(f"{fio_jobs_path}/{job_output}")
    data = json.load(f)
    job_outputs.append(data)

print(job_outputs[0])

## Cluster IICT
On souhaite déployer un container docker avec un PV dans le cluster IICT. Le container contient l'outil *fio* et un dossier *fio-jobs*.

Architecting-it a fait des [scripts](https://github.com/architectingit/k8sstorage/blob/main/perfraw.sh) pour réaliser ses tests. On va s'inspirer de leur script pour réaliser nos tests.
### Benchmarking
#### Connection au cluster IICT
Connectez-vous au VPN de l'école, puis:
```bash
kubectl config get-contexts         # is ` iict ` listed ?
kubectl config use-context iict
```

#### Manual
Si vous désirer allez plus rapidement, sautez à la section suivante (on applique une configuration dans le cluster et récupère les résultats):

```bash
kubectl apply -f iict-fio-benchmark-manual.yaml
```

Observons quel classe de stockage sont disponibles :
```bash
$ kubectl get storageclass
NAME                 PROVISIONER                                     RECLAIMPOLICY   VOLUMEBINDINGMODE   ALLOWVOLUMEEXPANSION   AGE
longhorn (default)   driver.longhorn.io                              Delete          Immediate           true                   170d
nfs-client           cluster.local/nfs-subdir-external-provisioner   Delete          Immediate           true                   114d
```

Nous allons donc utiliser un PVC de storageclass `longhorn`.

Ensuite, on déploie un container simple:

```bash
$ kubectl apply --namespace=mercado -f iict-fio-benchmark-manual.yaml
```

On vérifie que le container a bien démarré:
```bash
$ kubectl get pods --namespace=mercado
```

On ouvre une session interactive et on install fio:
```bash
$ kubectl exec --namespace=mercado -it $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}') -- /bin/sh
/ # apk add fio
/ # fio --version
$ exit # on sort de la session
```

On va copier un fio-job et un script permettant de run tous les tests:
```bash
# copy test script
$ kubectl --namespace=mercado cp docker/iict/run-all-jobs.sh $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/

# copy all jobs
$ kubectl --namespace=mercado cp docker/iict/fio-jobs/random-read.fio $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/
$ kubectl --namespace=mercado cp docker/iict/fio-jobs/random-readwrite.fio $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/
$ kubectl --namespace=mercado cp docker/iict/fio-jobs/random-write.fio $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/
$ kubectl --namespace=mercado cp docker/iict/fio-jobs/sequential-read.fio $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/
$ kubectl --namespace=mercado cp docker/iict/fio-jobs/sequential-readwrite.fio $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/
$ kubectl --namespace=mercado cp docker/iict/fio-jobs/sequential-write.fio $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/
```

Et on exécute les tests:
```bash
# ouvrir une session interactive
kubectl exec --namespace=mercado -it $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}') -- /bin/sh
sh run-all-jobs.sh
exit
```

On récupère tous les outputs:
```bash
kubectl --namespace=mercado cp $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/random-read.fio-output fio-jobs/iict/random-read.fio-output
kubectl --namespace=mercado cp $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/random-readwrite.fio-output fio-jobs/iict/random-readwrite.fio-output
kubectl --namespace=mercado cp $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/random-write.fio-output fio-jobs/iict/random-write.fio-output
kubectl --namespace=mercado cp $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/sequential-read.fio-output fio-jobs/iict/sequential-read.fio-output
kubectl --namespace=mercado cp $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/sequential-readwrite.fio-output fio-jobs/iict/sequential-readwrite.fio-output
kubectl --namespace=mercado cp $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/sequential-write.fio-output fio-jobs/iict/sequential-write.fio-output
```

Une fois que l'on a terminé, on nettoie:
```bash
kubectl delete -f iict-fio-benchmark-manual.yaml
```

#### Image

Nous allons déployer une image contenant les scripts de tests et un script d'exécution des tests en quelques lignes. On déploie :

```bash
$ kubectl apply -f iict-fio-benchmark.yaml
```

On peut observer l'exécution du container en consultant les logs (ou en allant sur l'UI rancher qui se rafraîchit automatiquement):
```bash
$ kubectl logs --follow $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}') --namespace=mercado
```

Une fois que les logs affiche `"All jobs done"`, alors on peut récupérer les output et nettoyer les ressources:

```bash
$ sh iict-fio-benchmark-get-output.sh
$ kubectl delete -f iict-fio-benchmark.yaml
```

Maintenant on va visualiser nos données:


In [ ]:
# from: https://stackoverflow.com/a/3207973
from os import listdir
from os.path import isfile, join

fio_jobs_path = 'docker/iict/fio-jobs'
job_outputs_filename = [f for f in listdir(fio_jobs_path) if isfile(join(fio_jobs_path, f))]

In [ ]:
import json

job_outputs = []

# Load the results
for job_output in job_outputs_filename:
    f = open(f"fio-jobs/iict/{job_output}-output")
    data = json.load(f)
    job_outputs.append(data)

print(job_outputs[0])


Note: [google](https://cloud.google.com/compute/docs/disks/benchmarking-pd-performance) persistent disk benchmark

##### Déploiement d'une nouvelle image

Pour mettre à jour l'image que le cluster peut utiliser pour benchmark (avec de nouveaux scénarios de tests ou des modifications), on tag un commit de ce repository avec, par exemple :

```bash
$ git tag # what tag can we use?
...
v0.1.4
v0.1.5
v0.1.6
v0.1.7

$ git tag v0.1.8
$ git push origin tag v0.1.8
```

On patiente un moment que l'image soit mise en ligne puis on peut réaliser notre benchmark.

**Note**: La github Action ne build que les images en `v0.*.*` pour le moment.

### Configuration de Longhorn a tester
* replicas
* data locality
* scheduling policy?

[I/O operations and replicas](https://longhorn.io/docs/1.1.1/concepts/#231-how-read-and-write-operations-work-for-replicas)

Dans un volume Longhorn, lorsque les opérations I/O sont faites sur une replica, ces opérations se font d'abord sur le live data. Si le bloc cherché n'est pas sur le live data, alors, il va cherché le bloc dans la snapshot la plus récente jusqu'à ce qu'il trouve la bonne snapshot. Une fois trouvé, un index est conservé pour se souvenir de la snapshot la plus récente contenant le bloc recherché.

L'index est conservé en mémoire et a une taille de 1 byte par block de taille 4K.

[taille effective des volumes/replicas/snapshot](https://longhorn.io/docs/1.1.1/volumes-and-nodes/volume-size/#an-example-that-helps-understand-volume-size-and-volume-actual-size)

Au départ, les volumes ne prennent pas de places. Mais après qu'un document soit supprimé du FS dans le volume, cela n'est pas répliqué au niveau bloc: *"The truth of the data#1 deletion is that the data#1 is marked as removed in the filesystem level (For example, inode deletion in ext4). Since Longhorn operates on the block level and does not understand the filesystem, as a result, the disk blocks/space storing data#1 won’t be released after the deletion."*

Est-ce que ce serait intéressant de démontrer ça?

[Longhorn engine par volume](https://longhorn.io/docs/1.1.1/concepts/#11-the-longhorn-manager-and-the-longhorn-engine)

Super intéressant.

[replica par défaut](https://longhorn.io/docs/1.1.1/references/settings/#default-replica-count)

3 apparament??? Est-ce que je peux le vérifier moi-même?

[volumeMode block](https://longhorn.io/docs/0.8.0/examples/block-volume/)

what is this

[PV avec csi option, numberofreplicas](https://longhorn.io/docs/0.8.0/examples/csi-pv/)

[volumeMode: Filesystem](https://github.com/longhorn/longhorn/blob/8318d99136e0d5b39f01a37adf9cae5342034d7e/examples/csi/example_pv.yaml#L8)

[persistentVolumeReclaimPolicy: Delete](https://github.com/longhorn/longhorn/blob/8318d99136e0d5b39f01a37adf9cae5342034d7e/examples/csi/example_pv.yaml#L11)

[volume mode](https://longhorn.io/docs/0.8.0/install/customizing-default-settings/#via-longhorn-deployment-yaml-file)

* filesystem (défaut)
* block

![Default replicas](/img/default-pvc-replicas.png)
3 replicas par défaut

Le type de volume [block](https://kubernetes.io/blog/2019/03/07/raw-block-volume-support-to-beta/#why-add-raw-block-volumes-to-kubernetes) est pour les applications spécialisées. Benchmarker un volume de ce type mène à une erreur ('Not a directory'). Est-ce possible de tester un raw block device avec fio ?


